In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import sys
import hashlib
import time
import shutil
import re

In [2]:
import PyPDF2
import nltk
import re
import numpy as np
from PyPDF2 import PdfFileWriter, PdfFileReader
from nltk import sent_tokenize,word_tokenize
from nltk.tokenize  import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.probability import FreqDist
from string import punctuation
from nltk.corpus import stopwords
from nltk.text import Text
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
chrome_options = Options()
chrome_options.add_extension(r"D:\RHYMcygiene\domain_monitoring\extension_1_5_4_0.crx")


driver = webdriver.Chrome(r"D:\RHYMcygiene\database\chromedriver.exe",options=chrome_options)
site='https://threats.kaspersky.com/en/vulnerability/'
driver.get(site)
driver.maximize_window()

In [22]:
source = driver.page_source
soup = BeautifulSoup(source)

In [23]:
text=soup.find('div',class_='clearfix js-print-area content-unmask direction-ltr').text

In [24]:
md5=re.findall(r"(^[a-fA-F\d]{32})", text)
sha1=re.findall(r"(^[a-fA-F\d]{44})", text)
sha256=re.findall(r"([a-fA-F\d]{64})", text)

In [25]:
md5

[]

In [26]:
sha1

[]

In [27]:
sha256

[]

In [42]:

user = input("Enter text here ")
h = hashlib.sha1(user.encode())
h2 = h.hexdigest()
print(h2)

Enter text here e4d909c290d0fb1ca068ffaddf22cbd0
24cacd2b74fd8557af7adfeee243ea8ee4c2487f


In [41]:
import hashlib
print(hashlib.md5(b"e4d909c290d0fb1ca068ffaddf22cbd0").hexdigest())
print(hashlib.sha1("Hello MD5".encode('utf-8')).hexdigest())

c25957d9fa131cc4940842696b735234
b986c26586db53671af46b27524979d9023a91a2


In [7]:
tokens=[]
for i in text.replace('\n','').split(' '):
    if len(i)>0:
        tokens.append(i)
text=' '.join(tokens)

In [14]:
sent=sent_tokenize(text)

In [12]:
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler


nlp = spacy.load('en_core_web_sm')
new_ruler = EntityRuler(nlp).from_disk(r"D:\Threat_intel_services\NER\SPACY\rule_corpus.jsonl")
#new_ruler1 = EntityRuler(nlp).from_disk("malware_corpus.jsonl")
nlp.add_pipe(new_ruler)
#nlp.add_pipe(new_ruler1)

In [13]:
doc1 = nlp("Virus is a Chinese threat group that has been healthcare 2015 to the 2nd Bureau of the People’s Liberation Army (PLA) General Staff Department’s (GSD) 3rd Department, commonly known by its Military Unit Cover Designator (MUCD) as Unit 61398.")
print([(ent.text, ent.label_) for ent in doc1.ents])

[('Virus', 'ORG'), ('Chinese', 'NORP'), ('healthcare', 'sector'), ('2015', 'DATE'), ('the 2nd Bureau of the People’s Liberation Army', 'ORG'), ('PLA', 'ORG'), ('General Staff Department', 'ORG'), ('GSD', 'ORG'), ('3rd Department', 'ORG'), ('Military Unit Cover Designator', 'ORG')]


In [15]:
df=pd.DataFrame({'Description':sent})

In [17]:
norp_main=[]
malware_main=[]
threat_main=[]
date_main=[]
text=[]
#ID=[]
org_main=[]
sector_main=[]
for i in df.iterrows():
    text.append(i[1]['Description'])
    #ID.append(i[1]['ID'])
    doc=nlp(i[1]['Description'])
    norp=[]
    malware=[]
    threat=[]
    date=[]
    org=[]
    sector=[]
    for ent in doc.ents:
        if (ent.label_=='NORP') | (ent.label_=='GPE'):
            norp.append(ent.text)
        if ent.label_=='MALWARE':
            malware.append(ent.text)
        if ent.label_=='threat_group':
            threat.append(ent.text)
        if ent.label_=='ORG':
            org.append(ent.text)
        if ent.label_=='DATE':
            date.append(ent.text)
        if ent.label_=='sector':
            sector.append(ent.text)
        
    norp_main.append(norp)
    malware_main.append(malware)
    threat_main.append(threat)
    date_main.append(date)
    org_main.append(org)
    sector_main.append(sector)

In [19]:
results=pd.DataFrame({'Description':text,'Country':norp_main,'Threat_group':threat_main,'Malware':malware_main,'Date':date_main,'Organisation':org_main,'Sectors':sector_main})

In [20]:
results

,Description,Country,Threat_group,Malware,Date,Organisation,Sectors
0,State of the Criminal Addresscrowdstrike.com/b...,[],[],[gain],[the past few months],[CrowdStrike],"[hospitality, services, organizations, entitie..."
1,These attacks have included criminal groups re...,[Russian],[],[MalumPOS],[],[Carbanak],[organizations]
2,So how are they so successful?,[],[],[],[],[],[]
3,Criminal organizations have learned to target ...,[],[],[target],[],[],[organizations]
4,"Specifically, these attackers have discovered ...",[],[],[exploit],[],[],"[business, security]"
5,A franchise business is a business in which th...,[],[],"[owned, owned]",[],[the umbrella corporation],"[business, business, business, retail, legal, ..."
6,"Therefore, information technology assets, secu...",[],[],[owned],[],[],"[information, security, security]"
7,This doesn’t mean franchisees have carte blanc...,[],[],[],[],"[carte blanche authority, PCI, PCI DSS]","[security, legal]"
8,"However, it does mean that corporate security ...",[],[],[],[],[],"[security, defense]"
9,"As a result of these visibility gaps, the comp...",[],[],"[exploit, network]",[],[],[]
